In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pointCollection as pc


In [ ]:
%matplotlib notebook


In [ ]:
thefile='/home/ben/git_repos/surfaceChange/ATL15.h5'
thefile='/Volumes/ice2/ben/ATL14_test/001/ATL15.h5'
V0={}
dV0={}
A0={}
for group in ['', '_10km','_20km', '_40km']:   
    D15=pc.grid.data().from_h5(thefile, group='height_change'+group)
    field='cell_area'+group
    A=getattr(D15, field)
    dh=getattr(D15, 'delta_h'+group)
    dhdt=getattr(D15, 'dhdt_lag1'+group)
    A[A>1.e16]=np.NaN
    A0[group]=np.nansum(A)
    V0[group] = np.array([np.nansum(A*dh[:,:,ii]) for ii in range(dh.shape[2])])
    dV0[group] = np.array([np.nansum(A*dhdt[:,:,ii]) for ii in range(dhdt.shape[2])])


In [ ]:
Acell = {'':1.e3**2, '_10km':1.e4**2, '_20km':2.e4**2, '_40km':4.e4**2}

{key:(A0[key]-A0[''])/Acell[key] for key in A0}

In [ ]:
dhdt.shape

In [ ]:
{key:V0[key]-V0[''] for key in V0}

In [ ]:
{key:dV0[key]-dV0[''] for key in V0}

In [ ]:
plt.figure(); 
plt.plot(dV0['']/np.nansum(A0['']))
plt.plot(dV0['_10km']/np.nansum(A0['_10km']))
plt.plot(dV0['_20km']/np.nansum(A0['_20km']))
plt.plot(dV0['_40km']/np.nansum(A0['_40km']))
plt.legend(['1km', '10km', '20km', '40km'])

In [ ]:
hfig=plt.figure();
hax=hfig.subplots(1, 4, sharex=True, sharey=True)
for ii, av in enumerate(A0.keys()):
    D15=pc.grid.data().from_h5(thefile, group='height_change'+av)
    temp=getattr(D15, 'cell_area'+av)
    temp[temp>1.e15]=np.NaN
    hax[ii].imshow(getattr(D15, 'cell_area'+av), extent=D15.extent, origin='lower')

In [ ]:
D15=pc.grid.data().from_h5(thefile, group='height_change'+group)
D15

In [ ]:
av

In [ ]:
! h5ls /home/ben/git_repos/surfaceChange/ATL15.h5/height_change_10km